In [1]:
import os

# Data Wrangling
import numpy as np
import pandas as pd

# Data Modeling
import statsmodels.api as sm          
import statsmodels.formula.api as smf # tilde formulas (y ~ x1 + x2)

# Graphing
import plotly.express as px         # Main interface
import plotly.figure_factory as ff  # Specialized plots (dendrograms, density plots)
import plotly.graph_objects as go   # Lower level interface

In [2]:
penguins = pd.read_csv('./penguins.csv')
penguins

Unnamed: 0    species     island  bill_length_mm  bill_depth_mm  \
0             1     Adelie  Torgersen            39.1           18.7   
1             2     Adelie  Torgersen            39.5           17.4   
2             3     Adelie  Torgersen            40.3           18.0   
3             4     Adelie  Torgersen             NaN            NaN   
4             5     Adelie  Torgersen            36.7           19.3   
..          ...        ...        ...             ...            ...   
339         340  Chinstrap      Dream            55.8           19.8   
340         341  Chinstrap      Dream            43.5           18.1   
341         342  Chinstrap      Dream            49.6           18.2   
342         343  Chinstrap      Dream            50.8           19.0   
343         344  Chinstrap      Dream            50.2           18.7   

     flipper_length_mm  body_mass_g     sex  year  
0                181.0       3750.0    male  2007  
1                186.0       3800.0  female  2007  
2                195.0       3250.0  female  2007  
3                  NaN          NaN     NaN  2007  
4                193.0       3450.0  female  2007  
..                 ...          ...     ...   ...  
339              207.0       4000.0    male  2009  
340              202.0       3400.0  female  2009  
341              193.0       3775.0    male  2009  
342              210.0       4100.0    male  2009  
343              198.0       3775.0  female  2009  

[344 rows x 9 columns]

# Express & Figure Factory

In [3]:
# Scatterplot
px.scatter(penguins, x = 'bill_length_mm', y = 'bill_depth_mm', color = 'species', trendline="ols")
# px._____(penguins, _____, _____, _____, trendline="ols")

In [4]:
# Distribution Plot
species   = ['Adelie', 'Gentoo', 'Chinstrap']
hist_data = penguins.loc[(penguins.flipper_length_mm.notna()), ['species', 'flipper_length_mm']].copy()
hist_data = [hist_data.loc[(hist_data.species == sp), 'flipper_length_mm'] for sp in species]

fig = ff.create_distplot(hist_data, group_labels = species, bin_size=.2)
# fig = ff._____(hist_data, group_labels = _____, bin_size=.2)
fig


In [5]:
# Scatterplot (3d)
fig = px.scatter_3d(penguins, x = 'bill_length_mm', y = 'bill_depth_mm', z = 'flipper_length_mm', color = 'species')
# px._____(penguins, _____, _____, _____, , color = 'species')
fig 

# Customizing a Plot

In [6]:
# Recreate Scatterplot (3d) with transparancy and control over point color

color_mapping = {'Adelie':'#636EFA', 'Gentoo':'#EF553B', 'Chinstrap':'#00CC96'}
color_list    = [color_mapping[i] for i in penguins['species']]


fig = go.Figure(data=[go.Scatter3d(
    x = penguins['bill_length_mm'], 
    y = penguins['bill_depth_mm'], 
    z = penguins['flipper_length_mm'],
    
    mode='markers',
    marker=dict(
        color= color_list, 
        opacity=0.6 
    )
)])

# fig = go.Figure(data=[go.Scatter3d(
#     x = penguins['bill_length_mm'], 
#     y = penguins['bill_depth_mm'], 
#     z = penguins['flipper_length_mm'],
    
#     mode='markers',
#     marker=dict(
#         color= _____, 
#         opacity= _____ 
#     )
# )])


# fig.update_layout(margin=dict(l=0, r=0, b=0, t=0)) # zoom camera
fig.update_layout(scene = dict(
    xaxis_title='bill_length_mm',
    yaxis_title='bill_depth_mm',
    zaxis_title='flipper_length_mm'))
fig.show()



# Adding a Fitted Plane (Demo)

In [7]:
# Model fitting:
penguins_nona = penguins[['species', 'bill_length_mm', 'bill_depth_mm', 'flipper_length_mm']].dropna()
fm = smf.ols(formula='flipper_length_mm ~ species + bill_length_mm + bill_depth_mm', data=penguins_nona)

res = fm.fit()

print(res.summary())

                            OLS Regression Results                            
Dep. Variable:      flipper_length_mm   R-squared:                       0.847
Model:                            OLS   Adj. R-squared:                  0.845
Method:                 Least Squares   F-statistic:                     464.7
Date:                Mon, 23 Sep 2024   Prob (F-statistic):          9.97e-136
Time:                        17:47:08   Log-Likelihood:                -1068.3
No. Observations:                 342   AIC:                             2147.
Df Residuals:                     337   BIC:                             2166.
Df Model:                           4                                         
Covariance Type:            nonrobust                                         
                           coef    std err          t      P>|t|      [0.025      0.975]
----------------------------------------------------------------------------------------
Intercept              127.6915 

In [8]:
# Model parameter
res.params

Intercept               127.691468
species[T.Chinstrap]     -1.306232
species[T.Gentoo]        27.564848
bill_length_mm            0.700442
bill_depth_mm             1.912699
dtype: float64

In [9]:
# Transparant predictions 
# Note: relying on a global variable like this is bad practice.
def predict_flipper_len(species, bill_length_mm, bill_depth_mm):
    yHat = res.params['Intercept']

    if species == 'Chinstrap':
        yHat += res.params['species[T.Chinstrap]']
    elif species == 'Gentoo':
        yHat += res.params['species[T.Gentoo]']

    yHat += res.params['bill_length_mm'] * bill_length_mm
    yHat += res.params['bill_depth_mm'] * bill_depth_mm

    return(yHat)


def mk_plane(species = 'Adelie'):

    # get corners of the plane to define
    min_length = penguins_nona.loc[penguins_nona['species'] == species, 'bill_length_mm'].min()
    max_length = penguins_nona.loc[penguins_nona['species'] == species, 'bill_length_mm'].max()

    min_depth  = penguins_nona.loc[penguins_nona['species'] == species, 'bill_depth_mm'].min()
    max_depth  = penguins_nona.loc[penguins_nona['species'] == species, 'bill_depth_mm'].max()

    temp = pd.DataFrame({'bill_length_mm' : [min_length, min_length, max_length, max_length],
                         'bill_depth_mm'  : [min_depth, max_depth, min_depth, max_depth]})

    temp['species'] = species
    temp['flipper_length_mm'] = np.nan

    # calculate the z for each
    for i in temp.index:
        temp.loc[i, 'flipper_length_mm'] = predict_flipper_len(species = temp.loc[i, 'species'], 
                                                               bill_length_mm = temp.loc[i, 'bill_length_mm'], 
                                                               bill_depth_mm = temp.loc[i, 'bill_depth_mm'])
    return(temp)

In [10]:
fig

In [11]:
for sp in ['Adelie', 'Gentoo', 'Chinstrap']:

    prediction_plane = mk_plane(species = sp)

    fig = fig.add_trace(go.Mesh3d(
        x = prediction_plane['bill_length_mm'], 
        y = prediction_plane['bill_depth_mm'], 
        z = prediction_plane['flipper_length_mm'],
        color = color_mapping[sp],
        opacity = 0.28))

    # fig = fig.add_trace(go._____(
    #     x = prediction_plane['bill_length_mm'], 
    #     y = prediction_plane['bill_depth_mm'], 
    #     z = prediction_plane['flipper_length_mm'],
    #     color = color_mapping[sp],
    #     opacity = 0.28))

fig